<a href="https://colab.research.google.com/github/Kovacs-Tibor-Benedek0515/Collab-Electricity-Models/blob/main/MDAMODEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn as sk
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, f1_score, classification_report, mean_absolute_error
from sklearn.model_selection import train_test_split, GridSearchCV, TimeSeriesSplit
from sklearn.linear_model import HuberRegressor, LinearRegression
import xgboost as xgb
from xgboost import XGBClassifier, XGBRegressor
import joblib
from datetime import datetime as dt

In [2]:
from google.colab import files
uploaded = files.upload()

Saving MDA_06_07_08_11_HU_GENERATION_M.xlsx to MDA_06_07_08_11_HU_GENERATION_M.xlsx
Saving MDA_06_07_08_11_AT_GENERATION.xlsx to MDA_06_07_08_11_AT_GENERATION (4).xlsx
Saving MDA_06_07_08_11_PRICE_DATA_M.xlsx to MDA_06_07_08_11_PRICE_DATA_M.xlsx
Saving DE_GEN_06_07_08_10.xlsx to DE_GEN_06_07_08_10 (4).xlsx
Saving MDA_06_07_08_11_ConsumptionData.xlsx to MDA_06_07_08_11_ConsumptionData (4).xlsx
Saving forecast-fuelmix-fuel.csv to forecast-fuelmix-fuel (4).csv
Saving latest-sensitivities-combined.csv to latest-sensitivities-combined (4).csv
Saving MDA_06_07_08_11_CBC_AUC_TO_HU.xlsx to MDA_06_07_08_11_CBC_AUC_TO_HU (4).xlsx
Saving MDA_06_07_08_11_CBC_TO_HU.xlsx to MDA_06_07_08_11_CBC_TO_HU (4).xlsx
Saving MDA_06_07_08_11_SK_GENERATION.xlsx to MDA_06_07_08_11_SK_GENERATION (5).xlsx
Saving MDA_06_07_08_11_SOLAR_WIND.xlsx to MDA_06_07_08_11_SOLAR_WIND (4).xlsx


In [3]:
cbc_auc= pd.read_excel('MDA_06_07_08_11_CBC_AUC_TO_HU.xlsx')
consumption = pd.read_excel('MDA_06_07_08_11_ConsumptionData.xlsx')
price = pd.read_excel('MDA_06_07_08_11_PRICE_DATA_M.xlsx')
de_generation = pd.read_excel('DE_GEN_06_07_08_10.xlsx')
at_generation = pd.read_excel('MDA_06_07_08_11_AT_GENERATION.xlsx')
sk_generation = pd.read_excel('MDA_06_07_08_11_SK_GENERATION.xlsx')
hu_generation = pd.read_excel('MDA_06_07_08_11_HU_GENERATION_M.xlsx')
cbc_to_hun = pd.read_excel('MDA_06_07_08_11_CBC_TO_HU.xlsx')
solar_wind = pd.read_excel('MDA_06_07_08_11_SOLAR_WIND.xlsx')
sensitivities = pd.read_csv('latest-sensitivities-combined.csv')
hun_fuelmix = pd.read_csv('forecast-fuelmix-fuel.csv')

## CBC_AUC TÁBLA

In [4]:
cbc_auc.drop(0, inplace=True)

In [5]:
cbc_auc.columns

Index(['Unnamed: 0', 'Unnamed: 1',
       'Filter: Filter-1\nSource: AT\nSink: HU\nPosition: M+1\nPrice Unit: EUR/MWh',
       'Unnamed: 3',
       'Filter: Filter-2\nSource: HR\nSink: HU\nPosition: M+1\nPrice Unit: EUR/MWh',
       'Unnamed: 5',
       'Filter: Filter-3\nSource: RO\nSink: HU\nPosition: M+1\nPrice Unit: EUR/MWh',
       'Unnamed: 7',
       'Filter: Filter-4\nSource: RS\nSink: HU\nPosition: M+1\nPrice Unit: EUR/MWh',
       'Unnamed: 9',
       'Filter: Filter-5\nSource: SI\nSink: HU\nPosition: M+1\nPrice Unit: EUR/MWh',
       'Unnamed: 11',
       'Filter: Filter-6\nSource: SK\nSink: HU\nPosition: M+1\nPrice Unit: EUR/MWh',
       'Unnamed: 13'],
      dtype='object')

In [6]:
cbc_auc

,Unnamed: 0,Unnamed: 1,Filter: Filter-1\nSource: AT\nSink: HU\nPosition: M+1\nPrice Unit: EUR/MWh,Unnamed: 3,Filter: Filter-2\nSource: HR\nSink: HU\nPosition: M+1\nPrice Unit: EUR/MWh,Unnamed: 5,Filter: Filter-3\nSource: RO\nSink: HU\nPosition: M+1\nPrice Unit: EUR/MWh,Unnamed: 7,Filter: Filter-4\nSource: RS\nSink: HU\nPosition: M+1\nPrice Unit: EUR/MWh,Unnamed: 9,Filter: Filter-5\nSource: SI\nSink: HU\nPosition: M+1\nPrice Unit: EUR/MWh,Unnamed: 11,Filter: Filter-6\nSource: SK\nSink: HU\nPosition: M+1\nPrice Unit: EUR/MWh,Unnamed: 13
1,2025-06-07 00:00:00,1,2025-05-14 00:00:00,21.1,2025-05-13 00:00:00,3.88,2025-05-14 00:00:00,2.81,2025-05-16 00:00:00,1.25,2025-05-14 00:00:00,5.41,2025-05-14 00:00:00,6.77
2,2025-06-07 00:00:00,2,2025-05-14 00:00:00,21.1,2025-05-13 00:00:00,3.88,2025-05-14 00:00:00,2.81,2025-05-16 00:00:00,1.25,2025-05-14 00:00:00,5.41,2025-05-14 00:00:00,6.77
3,2025-06-07 00:00:00,3,2025-05-14 00:00:00,21.1,2025-05-13 00:00:00,3.88,2025-05-14 00:00:00,2.81,2025-05-16 00:00:00,1.25,2025-05-14 00:00:00,5.41,2025-05-14 00:00:00,6.77
4,2025-06-07 00:00:00,4,2025-05-14 00:00:00,21.1,2025-05-13 00:00:00,3.88,2025-05-14 00:00:00,2.81,2025-05-16 00:00:00,1.25,2025-05-14 00:00:00,5.41,2025-05-14 00:00:00,6.77
5,2025-06-07 00:00:00,5,2025-05-14 00:00:00,21.1,2025-05-13 00:00:00,3.88,2025-05-14 00:00:00,2.81,2025-05-16 00:00:00,1.25,2025-05-14 00:00:00,5.41,2025-05-14 00:00:00,6.77
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1557,2025-08-10 00:00:00,21,2025-07-16 00:00:00,21.07,2025-07-15 00:00:00,3.16,2025-07-15 00:00:00,3.11,2025-07-11 00:00:00,1.55,2025-07-16 00:00:00,3.2,2025-07-16 00:00:00,6.78
1558,2025-08-10 00:00:00,22,2025-07-16 00:00:00,21.07,2025-07-15 00:00:00,3.16,2025-07-15 00:00:00,3.11,2025-07-11 00:00:00,1.55,2025-07-16 00:00:00,3.2,2025-07-16 00:00:00,6.78
1559,2025-08-10 00:00:00,23,2025-07-16 00:00:00,21.07,2025-07-15 00:00:00,3.16,2025-07-15 00:00:00,3.11,2025-07-11 00:00:00,1.55,2025-07-16 00:00:00,3.2,2025-07-16 00:00:00,6.78
1560,2025-08-10 00:00:00,24,2025-07-16 00:00:00,21.07,2025-07-15 00:00:00,3.16,2025-07-15 00:00:00,3.11,2025-07-11 00:00:00,1.55,2025-07-16 00:00:00,3.2,2025-07-16 00:00:00,6.78


In [7]:
cbc_auc.drop(columns=['Filter: Filter-1\nSource: AT\nSink: HU\nPosition: M+1\nPrice Unit: EUR/MWh', 'Filter: Filter-2\nSource: HR\nSink: HU\nPosition: M+1\nPrice Unit: EUR/MWh', 'Filter: Filter-3\nSource: RO\nSink: HU\nPosition: M+1\nPrice Unit: EUR/MWh', 'Filter: Filter-4\nSource: RS\nSink: HU\nPosition: M+1\nPrice Unit: EUR/MWh', 'Filter: Filter-5\nSource: SI\nSink: HU\nPosition: M+1\nPrice Unit: EUR/MWh', 'Filter: Filter-6\nSource: SK\nSink: HU\nPosition: M+1\nPrice Unit: EUR/MWh'], inplace=True)

In [8]:
cbc_auc.rename(columns={'Unnamed: 0': 'Day',
                    'Unnamed: 1': 'Hour',
                    'Unnamed: 3': 'AT-HU',
                    'Unnamed: 5': 'HR-HU',
                    'Unnamed: 7': 'RO-HU',
                    'Unnamed: 9': 'RS-HU',
                    'Unnamed: 11': 'SI-HU',
                    'Unnamed: 13': 'SK-HU'}, inplace=True)

In [9]:
cbc_auc['Date (CET)'] = pd.to_datetime(cbc_auc['Day']) + pd.to_timedelta(cbc_auc['Hour'] - 1, unit='h')

In [10]:
cbc_auc.drop(columns=['Day', 'Hour'], inplace=True)

In [11]:
cbc_auc = cbc_auc[['Date (CET)', 'AT-HU', 'HR-HU', 'RO-HU', 'RS-HU', 'SI-HU', 'SK-HU']]

In [12]:
cbc_auc

,Date (CET),AT-HU,HR-HU,RO-HU,RS-HU,SI-HU,SK-HU
1,2025-06-07 00:00:00,21.1,3.88,2.81,1.25,5.41,6.77
2,2025-06-07 01:00:00,21.1,3.88,2.81,1.25,5.41,6.77
3,2025-06-07 02:00:00,21.1,3.88,2.81,1.25,5.41,6.77
4,2025-06-07 03:00:00,21.1,3.88,2.81,1.25,5.41,6.77
5,2025-06-07 04:00:00,21.1,3.88,2.81,1.25,5.41,6.77
...,...,...,...,...,...,...,...
1557,2025-08-10 20:00:00,21.07,3.16,3.11,1.55,3.2,6.78
1558,2025-08-10 21:00:00,21.07,3.16,3.11,1.55,3.2,6.78
1559,2025-08-10 22:00:00,21.07,3.16,3.11,1.55,3.2,6.78
1560,2025-08-10 23:00:00,21.07,3.16,3.11,1.55,3.2,6.78


## CBC TO HUN TÁBLA

In [60]:
cbc_to_hun.columns

Index(['Unnamed: 0', 'Unnamed: 1',
       'Type: CBS\nFilter: Filter-1\nSource: AT\nSink: HU\nNetted: True',
       'Type: CBS\nFilter: Filter-2\nSource: HR\nSink: HU\nNetted: True',
       'Type: CBS\nFilter: Filter-3\nSource: RO\nSink: HU\nNetted: True',
       'Type: CBS\nFilter: Filter-4\nSource: RS\nSink: HU\nNetted: True',
       'Type: CBS\nFilter: Filter-5\nSource: SI\nSink: HU\nNetted: True',
       'Type: CBS\nFilter: Filter-6\nSource: SK\nSink: HU\nNetted: True'],
      dtype='object')

In [61]:
cbc_to_hun.drop(0, inplace=True)
cbc_to_hun.rename(columns={'Unnamed: 0' : 'Date (CET)',
                           "Type: CBS\nFilter: Filter-1\nSource: AT\nSink: HU\nNetted: True" : 'AT-HU',
                           "Type: CBS\nFilter: Filter-2\nSource: HR\nSink: HU\nNetted: True" : 'HR-HU',
                           "Type: CBS\nFilter: Filter-3\nSource: RO\nSink: HU\nNetted: True" : 'RO-HU',
                           "Type: CBS\nFilter: Filter-4\nSource: RS\nSink: HU\nNetted: True" : 'RS-HU',
                           "Type: CBS\nFilter: Filter-5\nSource: SI\nSink: HU\nNetted: True" : 'SI-HU',
                           "Type: CBS\nFilter: Filter-6\nSource: SK\nSink: HU\nNetted: True" : 'SK-HU'}, inplace=True)
cbc_to_hun.drop(columns=['Unnamed: 1'], inplace=True)

In [62]:
cbc_to_hun

,Date (CET),AT-HU,HR-HU,RO-HU,RS-HU,SI-HU,SK-HU
1,2025-06-07 00:00:00,1765,-299,-646,-235,417,1302
2,2025-06-08 00:00:00,2167,-402,-2056,-245,611,1976
3,2025-06-09 00:00:00,1831,-410,-1577,-625,376,1916
4,2025-06-10 00:00:00,1147,457,-1508,18,367,1034
5,2025-06-11 00:00:00,1862,-164,-1383,-427,404,1671
...,...,...,...,...,...,...,...
1557,2025-08-07 00:00:00,643,-288,365,160,261,975
1558,2025-08-08 00:00:00,1413,-289,-1032,211,373,1062
1559,2025-08-09 00:00:00,1296,-525,-124,161,303,1051
1560,2025-08-10 00:00:00,948,-265,131,59,210,1011


## CONSUMPTION TÁBLA

In [13]:
consumption.drop(0, inplace=True)

In [14]:
consumption.columns

Index(['Unnamed: 0', 'Unnamed: 1',
       'Product: 'SOLAR','WIND'\nPlatform: WATTSIGHT\nTerm: Base',
       'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7',
       'Unnamed: 8', 'Unnamed: 9',
       'Product: 'CONSUMPTION'\nPlatform: WATTSIGHT\nTerm: Base',
       'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14',
       'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17'],
      dtype='object')

In [15]:
consumption

,Unnamed: 0,Unnamed: 1,"Product: 'SOLAR','WIND'\nPlatform: WATTSIGHT\nTerm: Base",Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Product: 'CONSUMPTION'\nPlatform: WATTSIGHT\nTerm: Base,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17
1,2025-06-07 00:00:00,1,366,15700.5,85,43.81,1838.25,646.25,NaN,0,5891.59,44257.31,1814.18,5081.68,15291.24,4718.76,3142.39,2283.83
2,2025-06-07 00:00:00,2,587,15321,89,41.4,2027,524.75,NaN,0,5556.92,42772.2,1629.67,4696.85,14531.89,4551.77,2836.98,2176.26
3,2025-06-07 00:00:00,3,693,13917.25,87,38.42,2264.25,461.5,NaN,0,5417.5,41548.01,1522.45,4473.11,14122.71,4500.47,2646.56,2122.48
4,2025-06-07 00:00:00,4,754,13125.5,97,34.86,2778.25,406.25,NaN,0,5283.69,40699.46,1485.74,4388.77,13745.99,4469.04,2533.41,2088.17
5,2025-06-07 00:00:00,5,698,12075.56,84,37.14,3195.61,369,NaN,0,5271.07,40811.01,1459.66,4324.77,13474.51,4349.64,2526.97,2075.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1580,2025-08-11 00:00:00,20,NaN,2463.45,648.34,306.07,1186.1,926.42,NaN,0.77,7474.29,57692.01,NaN,NaN,NaN,NaN,NaN,NaN
1581,2025-08-11 00:00:00,21,NaN,3660.46,680.57,28.11,110.53,943.88,NaN,1.2,7549.12,57078.73,NaN,NaN,NaN,NaN,NaN,NaN
1582,2025-08-11 00:00:00,22,NaN,5183.29,699.38,32.76,0,1028.23,NaN,1.62,7247.61,55385.11,NaN,NaN,NaN,NaN,NaN,NaN
1583,2025-08-11 00:00:00,23,NaN,6443.27,701.87,31.16,0,1139.89,NaN,1.88,6820.22,52179.51,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
consumption_cols = []

for col in consumption.columns:
    col = col.strip()  # felesleges szóközök levágása
    col = col.replace("Actual('SOLAR','WIND')", "Actual_SOLAR_WIND")
    col = col.replace("Actual('CONSUMPTION')", "Actual_CONSUMPTION")
    col = col.replace("[", "_").replace("]", "").replace(" ", "_")
    col = col.replace("Profile_Date", "Profile_Date").replace("Hour", "Hour")
    consumption_cols.append(col)

consumption.columns = consumption_cols

# Eredmény ellenőrzése
print(consumption.columns.tolist())

['Unnamed:_0', 'Unnamed:_1', "Product:_'SOLAR','WIND'\nPlatform:_WATTSIGHT\nTerm:_Base", 'Unnamed:_3', 'Unnamed:_4', 'Unnamed:_5', 'Unnamed:_6', 'Unnamed:_7', 'Unnamed:_8', 'Unnamed:_9', "Product:_'CONSUMPTION'\nPlatform:_WATTSIGHT\nTerm:_Base", 'Unnamed:_11', 'Unnamed:_12', 'Unnamed:_13', 'Unnamed:_14', 'Unnamed:_15', 'Unnamed:_16', 'Unnamed:_17']


In [17]:
consumption.drop(columns=['Unnamed:_1', "Product:_'SOLAR','WIND'\nPlatform:_WATTSIGHT\nTerm:_Base", 'Unnamed:_3', 'Unnamed:_4', 'Unnamed:_5', 'Unnamed:_6', 'Unnamed:_7', 'Unnamed:_8', 'Unnamed:_9'], inplace=True)

In [18]:
consumption

,Unnamed:_0,Product:_'CONSUMPTION'\nPlatform:_WATTSIGHT\nTerm:_Base,Unnamed:_11,Unnamed:_12,Unnamed:_13,Unnamed:_14,Unnamed:_15,Unnamed:_16,Unnamed:_17
1,2025-06-07 00:00:00,5891.59,44257.31,1814.18,5081.68,15291.24,4718.76,3142.39,2283.83
2,2025-06-07 00:00:00,5556.92,42772.2,1629.67,4696.85,14531.89,4551.77,2836.98,2176.26
3,2025-06-07 00:00:00,5417.5,41548.01,1522.45,4473.11,14122.71,4500.47,2646.56,2122.48
4,2025-06-07 00:00:00,5283.69,40699.46,1485.74,4388.77,13745.99,4469.04,2533.41,2088.17
5,2025-06-07 00:00:00,5271.07,40811.01,1459.66,4324.77,13474.51,4349.64,2526.97,2075.19
...,...,...,...,...,...,...,...,...,...
1580,2025-08-11 00:00:00,7474.29,57692.01,NaN,NaN,NaN,NaN,NaN,NaN
1581,2025-08-11 00:00:00,7549.12,57078.73,NaN,NaN,NaN,NaN,NaN,NaN
1582,2025-08-11 00:00:00,7247.61,55385.11,NaN,NaN,NaN,NaN,NaN,NaN
1583,2025-08-11 00:00:00,6820.22,52179.51,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
consumption.rename(columns={'Unnamed:_0' : 'Date (CET)',
                          "Product:_'CONSUMPTION'\nPlatform:_WATTSIGHT\nTerm:_Base": 'AT',
                          'Unnamed:_11': 'DE',
                          'Unnamed:_12': 'HR',
                          'Unnamed:_13': 'HU',
                          'Unnamed:_14': 'PL',
                          'Unnamed:_15': 'RO',
                          'Unnamed:_16': 'RS',
                          'Unnamed:_17': 'SK'}, inplace=True)

In [20]:
consumption

,Date (CET),AT,DE,HR,HU,PL,RO,RS,SK
1,2025-06-07 00:00:00,5891.59,44257.31,1814.18,5081.68,15291.24,4718.76,3142.39,2283.83
2,2025-06-07 00:00:00,5556.92,42772.2,1629.67,4696.85,14531.89,4551.77,2836.98,2176.26
3,2025-06-07 00:00:00,5417.5,41548.01,1522.45,4473.11,14122.71,4500.47,2646.56,2122.48
4,2025-06-07 00:00:00,5283.69,40699.46,1485.74,4388.77,13745.99,4469.04,2533.41,2088.17
5,2025-06-07 00:00:00,5271.07,40811.01,1459.66,4324.77,13474.51,4349.64,2526.97,2075.19
...,...,...,...,...,...,...,...,...,...
1580,2025-08-11 00:00:00,7474.29,57692.01,NaN,NaN,NaN,NaN,NaN,NaN
1581,2025-08-11 00:00:00,7549.12,57078.73,NaN,NaN,NaN,NaN,NaN,NaN
1582,2025-08-11 00:00:00,7247.61,55385.11,NaN,NaN,NaN,NaN,NaN,NaN
1583,2025-08-11 00:00:00,6820.22,52179.51,NaN,NaN,NaN,NaN,NaN,NaN


## DE TERMELÉS TÁBLA

In [25]:
de_generation_col_list = de_generation.columns.tolist()

In [26]:
de_generation = de_generation.drop(columns=de_generation_col_list[14:])

In [27]:
de_generation

,Profile Date,Hour,BIOMASS,FOSSIL GAS,FOSSIL COAL,FOSSIL OIL,GEOTHERMAL,HYDRO,LIGNITE,OTHER,SOLAR,WASTE,WIND,TOTAL
0,2025-06-07,1,7881.00,7609.00,2069.25,717.50,42,11534.00,9913.00,427.0,0.00,1999.25,31459.75,73651.75
1,2025-06-07,2,7887.00,7134.00,1850.00,716.75,42,8528.75,10491.00,427.0,0.00,1997.50,30674.25,69748.25
2,2025-06-07,3,7901.75,7086.00,1563.50,716.75,42,8403.25,10678.50,427.0,0.00,2004.00,27883.75,66706.50
3,2025-06-07,4,7869.50,6929.50,1536.75,716.50,42,7855.25,10660.75,427.0,0.00,2006.75,26299.75,64343.75
4,2025-06-07,5,7869.75,7021.25,1787.25,716.75,42,5573.25,10710.00,427.0,11.50,1993.00,24197.75,60349.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1555,2025-08-10,20,7664.00,9424.25,1212.75,552.00,44,11839.50,12571.75,441.0,10643.25,1354.50,11978.50,67725.50
1556,2025-08-10,21,7783.00,11418.00,1673.25,552.00,44,15407.50,14243.50,441.0,1564.50,1356.25,9956.25,64439.25
1557,2025-08-10,22,7850.00,12086.00,1810.25,552.00,44,13668.00,14452.75,441.0,1.75,1355.75,10475.00,62736.50
1558,2025-08-10,23,7885.25,12200.75,1709.00,552.00,44,9853.25,14753.00,441.0,0.00,1381.25,11930.75,60750.25


In [28]:
de_generation.rename(columns={'Unnamed: 0': 'Day',
                              'Unnamed: 1': 'Hour',
                              "Country: 'DE-Amprion'" : '[DE-Ampirion] Biomass',
                              'Unnamed: 3': '[DE-Ampirion] Fossil Coal Derived Gas',
                              'Unnamed: 4': '[DE-Ampirion] Fossil Gas',
                              'Unnamed: 5': '[DE-Ampirion] Fossil Hard Coal',
                              'Unnamed: 6': '[DE-Ampirion] Fossil Oil',
                              'Unnamed: 7': '[DE-Ampirion] Geothermal',
                              'Unnamed: 8': '[DE-Ampirion] Hydro Pumped Storage Consumption',
                              'Unnamed: 9': '[DE-Amprion] Hydro Pumped Storage Production',
                              'Unnamed: 10': '[DE-Amprion] Hydro Run-of-river and poundage',
                              'Unnamed: 11': '[DE-Amprion] Hydro Water Reservoir',
                              'Unnamed: 12': '[DE-Amprion] Lignite',
                              'Unnamed: 13': '[DE-Amprion] Other',
                              'Unnamed: 14': '[DE-Amprion] Other renweable',
                              'Unnamed: 15': '[DE-Amprion] Solar',
                              'Unnamed: 16': '[DE-Amprion] Waste',
                              'Unnamed: 17': '[DE-Amprion] Wind Onshore'}, inplace=True)

In [29]:
de_generation

,Profile Date,Hour,BIOMASS,FOSSIL GAS,FOSSIL COAL,FOSSIL OIL,GEOTHERMAL,HYDRO,LIGNITE,OTHER,SOLAR,WASTE,WIND,TOTAL
0,2025-06-07,1,7881.00,7609.00,2069.25,717.50,42,11534.00,9913.00,427.0,0.00,1999.25,31459.75,73651.75
1,2025-06-07,2,7887.00,7134.00,1850.00,716.75,42,8528.75,10491.00,427.0,0.00,1997.50,30674.25,69748.25
2,2025-06-07,3,7901.75,7086.00,1563.50,716.75,42,8403.25,10678.50,427.0,0.00,2004.00,27883.75,66706.50
3,2025-06-07,4,7869.50,6929.50,1536.75,716.50,42,7855.25,10660.75,427.0,0.00,2006.75,26299.75,64343.75
4,2025-06-07,5,7869.75,7021.25,1787.25,716.75,42,5573.25,10710.00,427.0,11.50,1993.00,24197.75,60349.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1555,2025-08-10,20,7664.00,9424.25,1212.75,552.00,44,11839.50,12571.75,441.0,10643.25,1354.50,11978.50,67725.50
1556,2025-08-10,21,7783.00,11418.00,1673.25,552.00,44,15407.50,14243.50,441.0,1564.50,1356.25,9956.25,64439.25
1557,2025-08-10,22,7850.00,12086.00,1810.25,552.00,44,13668.00,14452.75,441.0,1.75,1355.75,10475.00,62736.50
1558,2025-08-10,23,7885.25,12200.75,1709.00,552.00,44,9853.25,14753.00,441.0,0.00,1381.25,11930.75,60750.25


## SK TERMELÉS TÁBLA

In [64]:
sk_generation.drop(0, inplace=True)
sk_generation.rename(columns={'Unnamed: 0': 'Date (CET)',
                              "Country: 'SK'": 'BIOMASS',
                              'Unnamed: 3': 'FOSSIL GAS',
                              'Unnamed: 4': 'FOSSIL HARD COAL',
                              'Unnamed: 5': 'FOSSIL OIL',
                              'Unnamed: 6': 'HYDRO',
                              'Unnamed: 10': 'LIGNITE',
                              'Unnamed: 11' : 'NUCLEAR',
                              'Unnamed: 12': 'OTHER',
                              'Unnamed: 14': 'SOLAR',
                              'Unnamed: 15': 'Wind Onshore'}, inplace=True)
sk_generation['HYDRO'] = sk_generation['HYDRO'] + sk_generation['Unnamed: 7'] + sk_generation['Unnamed: 8'] + sk_generation['Unnamed: 9']
sk_generation['OTHER'] = sk_generation['OTHER'] + sk_generation['Unnamed: 13']
sk_generation.drop(columns=['Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 13'], inplace=True)

In [65]:
sk_generation

,Date (CET),Unnamed: 1,BIOMASS,FOSSIL GAS,FOSSIL HARD COAL,FOSSIL OIL,HYDRO,LIGNITE,NUCLEAR,OTHER,SOLAR,Wind Onshore
1,2025-06-07 00:00:00,1,96,129,0,39,NaN,0,1908,109,0,0
2,2025-06-07 00:00:00,2,95,124,0,39,NaN,0,1910,108,0,0
3,2025-06-07 00:00:00,3,95,129,0,39,NaN,0,1913,112,0,0
4,2025-06-07 00:00:00,4,97,121,0,38,NaN,0,1915,107,0,0
5,2025-06-07 00:00:00,5,95,130,0,38,NaN,0,1915,109,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1556,2025-08-10 00:00:00,20,105,114,0,45,NaN,0,1880,111,8,1
1557,2025-08-10 00:00:00,21,106,128,0,45,NaN,0,1889,119,1,1
1558,2025-08-10 00:00:00,22,106,112,0,45,NaN,0,1896,112,0,1
1559,2025-08-10 00:00:00,23,104,102,0,44,NaN,0,1899,111,0,1


### AT TERMELÉS TÁBLA

In [30]:
at_generation

,Unnamed: 0,Unnamed: 1,Country: 'AT',Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
0,Profile Date,Hour,[AT] Biomass,[AT] Fossil Gas,[AT] Fossil Hard Coal,[AT] Fossil Oil,[AT] Geothermal,[AT] Hydro Pumped Storage Consumption,[AT] Hydro Pumped Storage Production,[AT] Hydro Run-of-river and poundage,[AT] Hydro Water Reservoir,[AT] Other,[AT] Other renweable,[AT] Solar,[AT] Waste,[AT] Wind Onshore
1,2025-06-07 00:00:00,1,152,0,0,0,0,0,985.5,4329,693.75,22,0,0,100,366
2,2025-06-07 00:00:00,2,152,0,0,0,0,0,528.5,4411,491.5,22,0,0,100,587
3,2025-06-07 00:00:00,3,152,0,0,0,0,0,435.5,4471,520,22,0,0,100,693
4,2025-06-07 00:00:00,4,152,0,0,0,0,0,371.75,4460.25,533.5,22,0,0,100,754
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1556,2025-08-10 00:00:00,20,144,25.75,0,0,0,0,2354.5,3546.75,1034.75,22,0,158,100,1375
1557,2025-08-10 00:00:00,21,144,25.25,0,0,0,0,2458.75,3462,1054.25,22,0,7,100,1764
1558,2025-08-10 00:00:00,22,144,26,0,0,0,0,2484.75,3445.5,1065.75,22,0,0,100,1712
1559,2025-08-10 00:00:00,23,144,26,0,0,0,0,2433.75,3457.5,1056.25,22,0,0,100,1618


In [31]:
at_generation

,Unnamed: 0,Unnamed: 1,Country: 'AT',Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
0,Profile Date,Hour,[AT] Biomass,[AT] Fossil Gas,[AT] Fossil Hard Coal,[AT] Fossil Oil,[AT] Geothermal,[AT] Hydro Pumped Storage Consumption,[AT] Hydro Pumped Storage Production,[AT] Hydro Run-of-river and poundage,[AT] Hydro Water Reservoir,[AT] Other,[AT] Other renweable,[AT] Solar,[AT] Waste,[AT] Wind Onshore
1,2025-06-07 00:00:00,1,152,0,0,0,0,0,985.5,4329,693.75,22,0,0,100,366
2,2025-06-07 00:00:00,2,152,0,0,0,0,0,528.5,4411,491.5,22,0,0,100,587
3,2025-06-07 00:00:00,3,152,0,0,0,0,0,435.5,4471,520,22,0,0,100,693
4,2025-06-07 00:00:00,4,152,0,0,0,0,0,371.75,4460.25,533.5,22,0,0,100,754
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1556,2025-08-10 00:00:00,20,144,25.75,0,0,0,0,2354.5,3546.75,1034.75,22,0,158,100,1375
1557,2025-08-10 00:00:00,21,144,25.25,0,0,0,0,2458.75,3462,1054.25,22,0,7,100,1764
1558,2025-08-10 00:00:00,22,144,26,0,0,0,0,2484.75,3445.5,1065.75,22,0,0,100,1712
1559,2025-08-10 00:00:00,23,144,26,0,0,0,0,2433.75,3457.5,1056.25,22,0,0,100,1618


In [32]:
at_generation.drop(0, inplace=True)

In [69]:
at_generation.rename(columns={'Unnamed: 0' : 'Day',
                              'Unnamed: 1': 'Hour',
                              "Country: 'AT'": 'BIOMASS',
                              'Unnamed: 3': 'FOSSIL GAS',
                              'Unnamed: 4': 'FOSSIL HARD COAL',
                              'Unnamed: 5': 'FOSSIL OIL',
                              'Unnamed: 6': 'GEOTHERMAL',
                              'Unnamed: 7': 'HYDRO',
                              'Unnamed: 11': 'OTHER',
                              'Unnamed: 13': 'SOLAR',
                              'Unnamed: 14': 'Waste',
                              'Unnamed: 15': 'Wind Onshore'}, inplace=True)

In [35]:
at_generation

,Day,Hour,Country: 'AT',FOSSIL GAS,FOSSIL HARD COAL,FOSSIL OIL,GEOTHERMAL,HYDRO,Unnamed: 8,Unnamed: 9,Unnamed: 10,OTHER,Unnamed: 12,SOLAR,Waste,Wind Onshore
1,2025-06-07 00:00:00,1,152,0,0,0,0,0,985.5,4329,693.75,22,0,0,100,366
2,2025-06-07 00:00:00,2,152,0,0,0,0,0,528.5,4411,491.5,22,0,0,100,587
3,2025-06-07 00:00:00,3,152,0,0,0,0,0,435.5,4471,520,22,0,0,100,693
4,2025-06-07 00:00:00,4,152,0,0,0,0,0,371.75,4460.25,533.5,22,0,0,100,754
5,2025-06-07 00:00:00,5,148,0,0,0,0,0,460,4453.25,640,22,0,0,100,698
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1556,2025-08-10 00:00:00,20,144,25.75,0,0,0,0,2354.5,3546.75,1034.75,22,0,158,100,1375
1557,2025-08-10 00:00:00,21,144,25.25,0,0,0,0,2458.75,3462,1054.25,22,0,7,100,1764
1558,2025-08-10 00:00:00,22,144,26,0,0,0,0,2484.75,3445.5,1065.75,22,0,0,100,1712
1559,2025-08-10 00:00:00,23,144,26,0,0,0,0,2433.75,3457.5,1056.25,22,0,0,100,1618


In [36]:
at_generation['HYDRO'] = at_generation['HYDRO'] + at_generation['Unnamed: 8'] + at_generation['Unnamed: 9'] + at_generation['Unnamed: 10']
at_generation['OTHER'] = at_generation['OTHER'] + at_generation['Unnamed: 12']

In [37]:
at_generation.drop(columns=['Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 12'], inplace=True)

In [38]:
at_generation

,Day,Hour,Country: 'AT',FOSSIL GAS,FOSSIL HARD COAL,FOSSIL OIL,GEOTHERMAL,HYDRO,OTHER,SOLAR,Waste,Wind Onshore
1,2025-06-07 00:00:00,1,152,0,0,0,0,6008.25,22,0,100,366
2,2025-06-07 00:00:00,2,152,0,0,0,0,5431.0,22,0,100,587
3,2025-06-07 00:00:00,3,152,0,0,0,0,5426.5,22,0,100,693
4,2025-06-07 00:00:00,4,152,0,0,0,0,5365.5,22,0,100,754
5,2025-06-07 00:00:00,5,148,0,0,0,0,5553.25,22,0,100,698
...,...,...,...,...,...,...,...,...,...,...,...,...
1556,2025-08-10 00:00:00,20,144,25.75,0,0,0,6936.0,22,158,100,1375
1557,2025-08-10 00:00:00,21,144,25.25,0,0,0,6975.0,22,7,100,1764
1558,2025-08-10 00:00:00,22,144,26,0,0,0,6996.0,22,0,100,1712
1559,2025-08-10 00:00:00,23,144,26,0,0,0,6947.5,22,0,100,1618


## HU TERMELÉS TÁBLA

In [49]:
hu_generation.rename(columns={'Unnamed: 0': 'Day',
                              'Unnamed: 1': 'Hour',
                              "Country: 'HU'": 'BIOMASS',
                              'Unnamed: 3': 'FOSSIL GAS',
                              'Unnamed: 4': 'FOSSIL HARD COAL',
                              'Unnamed: 5': 'FOSSIL OIL',
                              'Unnamed: 6': 'GEOTHERMAL',
                              'Unnamed: 7': 'HYDRO',
                              'Unnamed: 9': 'LIGNITE',
                              'Unnamed: 10': 'NUCLEAR',
                              'Unnamed: 11': 'OTHER',
                              'Unnamed: 13': 'SOLAR',
                              'Unnamed: 14': 'Waste',
                              'Unnamed: 15': 'Wind Onshore'}, inplace=True)

In [41]:
hu_generation.drop(0, inplace=True)

In [42]:
hu_generation

,Day,Hour,Country: 'HU',FOSSIL GAS,FOSSIL HARD COAL,FOSSIL OIL,GEOTHERMAL,HYDRO,Unnamed: 8,LIGNITE,NUCLEAR,OTHER,Unnamed: 12,SOLAR,Waste,Wind Onshore
1,2025-06-07 00:00:00,1,181,510.25,40.5,0,1,15,25,132,1380,19.75,15,0,15,55.75
2,2025-06-07 00:00:00,2,179.75,251.5,40.75,0,1,15,25,133,1384.5,19.5,15.5,0,14.5,51
3,2025-06-07 00:00:00,3,181.25,123.25,40,0,1,15,25,124,1390.25,18.25,15,0,16,42.75
4,2025-06-07 00:00:00,4,180.75,120.25,39.5,0,1,15,25,123.5,1390.5,19,15.25,0,15.25,29.25
5,2025-06-07 00:00:00,5,181,129.25,40,0,1,15,25,123.25,1390.75,18.5,14,3.75,14.75,27.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1556,2025-08-10 00:00:00,20,116.75,414.25,34.25,0,0,8,5,153.75,1415,27,13,111.5,17,82.25
1557,2025-08-10 00:00:00,21,112.25,515,38.5,0,0,8,5,154,1414.5,31.25,15,16.5,17.75,80.75
1558,2025-08-10 00:00:00,22,113.5,476.25,38.75,0,0,8,5,154,1412,22,15,15.5,17.5,110.5
1559,2025-08-10 00:00:00,23,113.25,489.25,39.25,0,0,6.75,5,145.5,1410.5,16.75,13.25,5,16.75,136


In [45]:
hu_generation['HYDRO'] = hu_generation['HYDRO'] + hu_generation['Unnamed: 8']
hu_generation['OTHER'] = hu_generation['OTHER'] + hu_generation['Unnamed: 12']
hu_generation.drop(columns=['Unnamed: 8', 'Unnamed: 12'], inplace=True)

In [50]:
hu_generation

,Day,Hour,BIOMASS,FOSSIL GAS,FOSSIL HARD COAL,FOSSIL OIL,GEOTHERMAL,HYDRO,LIGNITE,NUCLEAR,OTHER,SOLAR,Waste,Wind Onshore
1,2025-06-07 00:00:00,1,181,510.25,40.5,0,1,40,132,1380,34.75,0,15,55.75
2,2025-06-07 00:00:00,2,179.75,251.5,40.75,0,1,40,133,1384.5,35.0,0,14.5,51
3,2025-06-07 00:00:00,3,181.25,123.25,40,0,1,40,124,1390.25,33.25,0,16,42.75
4,2025-06-07 00:00:00,4,180.75,120.25,39.5,0,1,40,123.5,1390.5,34.25,0,15.25,29.25
5,2025-06-07 00:00:00,5,181,129.25,40,0,1,40,123.25,1390.75,32.5,3.75,14.75,27.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1556,2025-08-10 00:00:00,20,116.75,414.25,34.25,0,0,13,153.75,1415,40,111.5,17,82.25
1557,2025-08-10 00:00:00,21,112.25,515,38.5,0,0,13,154,1414.5,46.25,16.5,17.75,80.75
1558,2025-08-10 00:00:00,22,113.5,476.25,38.75,0,0,13,154,1412,37,15.5,17.5,110.5
1559,2025-08-10 00:00:00,23,113.25,489.25,39.25,0,0,11.75,145.5,1410.5,30.0,5,16.75,136


### PRICE TÁBLA

In [52]:
price.rename(columns={'Price[EPEX AT]': 'AT PRICE',
                      'Volume[EPEX AT]' : 'AT VOLUME',
                      'Price[EPEX DE]': 'DE PRICE',
                      'Volume[EPEX DE]' : 'DE VOLUME',
                      'Price[HUPX]' : 'HU PRICE',
                      'Volume[HUPX]' : 'HU VOLUME',
                      'Price[OPCOM]' : 'RO PRICE',
                      'Volume[OPCOM]' : 'RO VOLUME',
                      'Price[OTE]' : 'PRICE OTE',
                      'Volume[OTE]' : 'SK VOLUME',
                      'Price[POLPX 1 EUR]' : 'PL PRICE',
                      'Volume[POLPX 1 EUR]' : 'PL VOLUME',
                      'Price[SEEPEX]' : 'RS PRICE',
                      'Volume[SEEPEX]' : 'RS VOLUME'}, inplace=True)

/tmp/ipython-input-2160207886.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price.rename(columns={'Price[EPEX AT]': 'AT PRICE',


In [53]:
price

,Date (CET),AT PRICE,AT VOLUME,DE PRICE,DE VOLUME,HU PRICE,HU VOLUME,RO PRICE,RO VOLUME,PRICE OTE,SK VOLUME,PL PRICE,PL VOLUME,RS PRICE,RS VOLUME
0,2025-06-07 00:00:00,80.00,1996.10,87.61,25795.20,101.56,3897.20,103.83,1851.00,111.48,1870.4,118.95,2913.00,120.00,372.20
1,2025-06-07 01:00:00,67.44,2015.50,83.50,26460.70,99.16,3639.90,99.50,1859.60,90.40,1817.6,118.95,2456.00,104.33,325.40
2,2025-06-07 02:00:00,64.14,2056.50,78.02,26996.10,96.25,3683.30,96.12,1685.60,86.28,1790.4,118.95,2425.00,100.10,498.60
3,2025-06-07 03:00:00,69.17,2053.20,81.89,26765.70,97.16,3565.10,96.24,1648.10,90.00,1809.1,114.02,2207.00,95.71,449.60
4,2025-06-07 04:00:00,73.85,2079.30,82.27,26409.30,91.98,3542.10,91.51,1773.40,94.54,1811.6,109.45,2012.00,96.73,430.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1556,2025-08-10 20:00:00,110.02,2010.90,118.95,27740.30,134.40,4661.00,138.32,1915.50,123.64,3900.7,120.80,3085.00,164.91,744.80
1557,2025-08-10 21:00:00,115.65,2307.90,122.45,27558.10,121.27,3969.30,121.04,1968.70,120.23,4002.7,121.37,3196.00,128.64,574.70
1558,2025-08-10 22:00:00,105.73,1297.10,106.98,27086.30,106.76,3533.00,106.75,1964.00,107.16,3864.9,114.61,2930.00,114.42,447.40
1559,2025-08-10 23:00:00,96.58,1151.40,98.90,27995.40,98.23,3240.30,98.21,1704.60,98.87,3764.5,102.66,2368.00,100.86,442.10


In [56]:
sensitivities.drop(0, inplace=True)

In [57]:
def clean_date_col(df):
  df['Date (CET)'] = df['Date (CET)'].replace(r'\+.*$', '', regex=True)
  df['Date (CET)'] = df['Date (CET)'].str.replace('[', '', regex=False)
  df['Date (CET)'] = df['Date (CET)'].str.replace(']', '', regex=False)
  df['Date (CET)'] = df['Date (CET)'].str.replace('/', '-', regex=False)
  return df

In [58]:
clean_date_col(sensitivities)

,Date (CET),Solar Upward Risk,Wind Upward Risk,Solar Downward Risk,Wind Downward Risk,Solar Outturn Deviation,Wind Outturn Deviation
1,07-06-2025 00:00,0,48,0,-40,0,-6
2,07-06-2025 01:00,0,45,0,-35,0,-6
3,07-06-2025 02:00,0,39,0,-30,0,-7
4,07-06-2025 03:00,0,39,0,-29,0,-13
5,07-06-2025 04:00,0,35,0,-23,4,-2
...,...,...,...,...,...,...,...
1556,10-08-2025 19:00,111,48,-20,-64,-6,-114
1557,10-08-2025 20:00,0,33,0,-59,17,-104
1558,10-08-2025 21:00,0,29,0,-60,16,-78
1559,10-08-2025 22:00,0,26,0,-62,5,-50


## HU FUEL MIX TÁBLA

In [67]:
hun_fuelmix.drop(0, inplace=True)
clean_date_col(hun_fuelmix)

,Date (CET),INTERCONNECTORS,INTERCONNECTORS SCHEDULED,OIL,OIL FORECAST,LIGNITE,LIGNITE FORECAST,HARD COAL,HARD COAL FORECAST,GAS,...,WASTE,WASTE FORECAST,GEOTHERMAL,GEOTHERMAL FORECAST,OTHER,OTHER FORECAST,NUCLEAR,NUCLEAR FORECAST,DEMAND,DEMAND FORECAST
1,07-06-2025 00:00,2299,2409,0,0,132,157,41,42,510,...,15,16,1,1,20,33,1380,1409,4685,4563
2,07-06-2025 01:00,2203,2319,0,0,133,143,41,42,252,...,15,15,1,1,20,21,1385,1382,4330,4247
3,07-06-2025 02:00,2133,2266,0,0,124,135,40,42,123,...,16,16,1,1,18,20,1390,1390,4124,4071
4,07-06-2025 03:00,2069,2084,0,0,124,134,40,41,120,...,15,16,1,1,19,20,1391,1391,4046,3970
5,07-06-2025 04:00,1996,2017,0,0,123,133,40,41,129,...,15,16,1,1,19,20,1391,1391,3987,3926
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1556,10-08-2025 19:00,2961,2970,0,0,154,158,34,35,414,...,17,17,0,0,27,40,1415,1415,5430,5279
1557,10-08-2025 20:00,3001,2945,0,0,154,158,39,38,515,...,18,17,0,0,31,33,1415,1415,5431,5330
1558,10-08-2025 21:00,2870,2769,0,0,154,158,39,40,476,...,18,18,0,0,22,29,1412,1413,5254,5142
1559,10-08-2025 22:00,2450,2445,0,0,146,154,39,40,489,...,17,18,0,0,17,22,1411,1411,4842,4785


## ADATTÍPUSOK

In [71]:
print(price.dtypes)
print(consumption.dtypes)
print(de_generation.dtypes)
print(at_generation.dtypes)
print(hu_generation.dtypes)
print(sk_generation.dtypes)
print(hun_fuelmix.dtypes)
print(sensitivities.dtypes)
print(cbc_to_hun.dtypes)
print(cbc_auc.dtypes)

Date (CET)    datetime64[ns]
AT PRICE             float64
AT VOLUME            float64
DE PRICE             float64
DE VOLUME            float64
HU PRICE             float64
HU VOLUME            float64
RO PRICE             float64
RO VOLUME            float64
PRICE OTE            float64
SK VOLUME            float64
PL PRICE             float64
PL VOLUME            float64
RS PRICE             float64
RS VOLUME            float64
dtype: object
Date (CET)    object
AT            object
DE            object
HR            object
HU            object
PL            object
RO            object
RS            object
SK            object
dtype: object
Profile Date    datetime64[ns]
Hour                     int64
BIOMASS                float64
FOSSIL GAS             float64
FOSSIL COAL            float64
FOSSIL OIL             float64
GEOTHERMAL               int64
HYDRO                  float64
LIGNITE                float64
OTHER                  float64
SOLAR                  float64
WASTE  